# Feature Engineering
Sean Wade

In [15]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [5]:
train = pd.read_csv('clean_data/train_eda.csv', index_col=0)
test = pd.read_csv('clean_data/test_eda.csv', index_col=0)

In [ ]:
train.AM

In [7]:
train.head()

,AMT_ANNUITY,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,APARTMENTS_AVG,BASEMENTAREA_AVG,...,EXT_SOURCE_2^3,EXT_SOURCE_2^2 EXT_SOURCE_3,EXT_SOURCE_2^2 DAYS_BIRTH,EXT_SOURCE_2 EXT_SOURCE_3^2,EXT_SOURCE_2 EXT_SOURCE_3 DAYS_BIRTH,EXT_SOURCE_2 DAYS_BIRTH^2,EXT_SOURCE_3^3,EXT_SOURCE_3^2 DAYS_BIRTH,EXT_SOURCE_3 DAYS_BIRTH^2,DAYS_BIRTH^3
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100006,29686.5,297000.0,135000.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,275132.312064,2264.326451,8.039498e+07,18.635304,661647.051121,2.349180e+10,0.153368,5445.325225,1.933364e+08,6.864416e+12
100014,21177.0,652500.0,112500.000,NaN,0.0,1.0,0.0,0.0,NaN,NaN,...,277040.296359,1546.671219,4.333456e+07,8.634816,241929.842302,6.778379e+09,0.048207,1350.655226,3.784259e+07,1.060272e+12
100015,10678.5,135000.0,38419.155,NaN,0.0,0.0,0.0,0.0,NaN,NaN,...,171138.756608,2012.534296,6.293480e+07,23.666727,740092.087827,2.314373e+10,0.278313,8703.234371,2.721625e+08,8.510906e+12
100017,28966.5,697500.0,225000.000,NaN,0.0,0.0,0.0,0.0,0.1474,0.0973,...,182187.833309,2474.872019,4.526897e+07,33.619103,614941.740282,1.124817e+10,0.456688,8353.478268,1.527971e+08,2.794879e+12
100018,32778.0,679500.0,189000.000,NaN,NaN,NaN,NaN,NaN,0.3495,0.1335,...,265475.776483,2211.029691,6.023702e+07,18.414683,501687.349942,1.366791e+10,0.153368,4178.330848,1.138339e+08,3.101277e+12


In [8]:
set(train.columns.tolist())

{'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'AMT_INCOME_TOTAL',
 'AMT_REQ_CREDIT_BUREAU_DAY',
 'AMT_REQ_CREDIT_BUREAU_HOUR',
 'AMT_REQ_CREDIT_BUREAU_MON',
 'AMT_REQ_CREDIT_BUREAU_QRT',
 'AMT_REQ_CREDIT_BUREAU_WEEK',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'CNT_CHILDREN',
 'CNT_FAM_MEMBERS',
 'CODE_GENDER',
 'COMMONAREA_MODE',
 'DAYS_BIRTH',
 'DAYS_BIRTH^2',
 'DAYS_BIRTH^3',
 'DAYS_EMPLOYED',
 'DAYS_EMPLOYED_ANOM',
 'DAYS_ID_PUBLISH',
 'DAYS_LAST_PHONE_CHANGE',
 'DAYS_REGISTRATION',
 'DEF_30_CNT_SOCIAL_CIRCLE',
 'DEF_60_CNT_SOCIAL_CIRCLE',
 'ELEVATORS_AVG',
 'EMERGENCYSTATE_MODE',
 'ENTRANCES_AVG',
 'EXT_SOURCE_1',
 'EXT_SOURCE_1 DAYS_BIRTH',
 'EXT_SOURCE_1 DAYS_BIRTH^2',
 'EXT_SOURCE_1 EXT_SOURCE_2',
 'EXT_SOURCE_1 EXT_SOURCE_2 DAYS_BIRTH',
 'EXT_SOURCE_1 EXT_SOURCE_2 EXT_SOURCE_3',
 'EXT_SOURCE_1 EXT_SOURCE_2^2',
 'EXT_SOURCE_1 EXT_SOURCE_3',
 'EXT_SOURCE_1 EXT_SOURCE_3 DAYS_BIRTH',
 'EXT_SOURCE_1 EXT_SOURCE_3^2',
 'EXT_SOURCE_1^2',
 'EXT_SOURCE_1^2 DAYS_BIRTH',
 'EXT_SOURCE_1^2 EXT_SOURCE_

In [20]:
import lightgbm as lgb

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

import gc

ModuleNotFoundError: No module named 'lightgbm'

In [18]:
def model(train, test, n_folds=5):
    train_ids = train.index
    test_ids = test.index
    
    labels = train['TARGET']
    
    features = train.drop(columns = 'TARGET')
    test_features = test
    
    cat_indices = 'auto'
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
     # Extract feature names
    feature_names = list(features.columns)
    
    # Convert to np arrays
    features = np.array(features)
    test_features = np.array(test_features)
    # Create the kfold object
    k_fold = KFold(n_splits = n_folds, shuffle = False, random_state = 50)
    
    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))
    
    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])
    
    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(features.shape[0])
    
    # Lists for recording validation and training scores
    valid_scores = []
    train_scores = []
    
    # Iterate through each fold
    for train_indices, valid_indices in k_fold.split(features):
        
        # Training data for the fold
        train_features, train_labels = features[train_indices], labels[train_indices]
        # Validation data for the fold
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
        # Create the model
        model = lgb.LGBMClassifier(n_estimators=10000, objective = 'binary', 
                                   class_weight = 'balanced', learning_rate = 0.05, 
                                   reg_alpha = 0.1, reg_lambda = 0.1, 
                                   subsample = 0.8, n_jobs = -1, random_state = 50)
        
        # Train the model
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'], categorical_feature = cat_indices,
                  early_stopping_rounds = 100, verbose = 200)
        
        # Record the best iteration
        best_iteration = model.best_iteration_
        
        # Record the feature importances
        feature_importance_values += model.feature_importances_ / k_fold.n_splits
        
        # Make predictions
        test_predictions += model.predict_proba(test_features, num_iteration = best_iteration)[:, 1] / k_fold.n_splits
        
        # Record the out of fold predictions
        out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]
        
        # Record the best score
        valid_score = model.best_score_['valid']['auc']
        train_score = model.best_score_['train']['auc']
        
        valid_scores.append(valid_score)
        train_scores.append(train_score)
        
        # Clean up memory
        gc.enable()
        del model, train_features, valid_features
        gc.collect()
        
    # Make the submission dataframe
    submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    
    # Make the feature importance dataframe
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
    # Overall validation score
    valid_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores}) 
    
    return submission, feature_importances, metrics
    

    
model(train, test)

Training Data Shape:  (75000, 294)
Testing Data Shape:  (25000, 294)


NameError: name 'KFold' is not defined